<a href="https://colab.research.google.com/github/GoeSoft/My_Python_Learn/blob/main/%D0%9D%D0%B5%D0%B9%D1%80%D0%BE_HR_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D1%81%D0%BE%D0%B1%D0%B5%D1%81%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai beautifulsoup4 lxml tiktoken

In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [3]:
import requests
from bs4 import BeautifulSoup

doc_url = "https://docs.google.com/document/d/14PDkow15jxyAXsm0l196aefbRPmpYd3v0iJe0sPupY8"

# Экспорт как HTML
export_url = doc_url + "/export?format=html"
response = requests.get(export_url)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')
interview_text = soup.get_text(separator="\n", strip=True)

print(f"✅ Загружено {len(interview_text)} символов.")

✅ Загружено 11078 символов.


In [4]:
from openai import OpenAI

client = OpenAI()

def analyze_interview(text):
    prompt = f"""
Вы — эксперт по подбору персонала (Нейро-HR). Проанализируйте следующее собеседование и подготовьте два отчёта.

КРИТЕРИИ ОЦЕНКИ:
1. Системность vs Суетология
2. Самостоятельность vs Зависимость
3. Ориентация на результат vs Ориентация на процесс
4. Эмоциональная устойчивость
5. Мотивация к развитию

Для КАЖДОГО критерия, где применимо (особенно для дихотомий: 1–3), сделайте следующее:
- Чётко определите, к какому полюсу склоняется кандидат.
- Приведите **аргументы «за»** отнесение к этому типу (с цитатами или перефразированием из диалога).
- Приведите **аргументы «против»** (если есть признаки противоположного полюса).
- Укажите **плюсы и минусы** такого профиля для работодателя.
- Если признаков недостаточно — так и напишите.

ОТЧЁТ 1: Краткий (в виде списка по критериям)
ОТЧЁТ 2: Подробный (с аргументацией, плюсами/минусами)

Текст собеседования:
{text}
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Вы — профессиональный HR-аналитик."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=2000
    )
    return response.choices[0].message.content.strip()

# Запуск анализа
full_report = analyze_interview(interview_text)
print(full_report)

### ОТЧЁТ 1: Краткий (в виде списка по критериям)

1. **Системность vs Суетология**
   - Полюс: Системность
   - Аргументы «за»: Кандидат упоминает о структурированности работы, о создании системы для упрощения работы рекрутеров.
   - Аргументы «против»: Непосредственно не выявлены.
   - Плюсы: Способность организовать рабочие процессы, улучшить эффективность.
   - Минусы: Возможная недостаточная гибкость в условиях быстро меняющейся среды.

2. **Самостоятельность vs Зависимость**
   - Полюс: Самостоятельность
   - Аргументы «за»: Кандидат самостоятельно принимал решения по улучшению процессов, искал новые источники для рекрутинга.
   - Аргументы «против»: Иногда зависела от мнения руководства (например, в вопросах увольнения).
   - Плюсы: Инициативность, способность к самостоятельной работе.
   - Минусы: Возможная недостаточная способность работать в условиях жесткого контроля.

3. **Ориентация на результат vs Ориентация на процесс**
   - Полюс: Ориентация на результат
   - Аргументы 